In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-03-13 23:39:17,669.669 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-03-13 23:39:17,669.669 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [1]:
import pyEX as p
import requests
import json
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
import itertools
import datetime
#import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
import pyspark.sql.window as py_w
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.requests import StockBarsRequest
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
from botocore.exceptions import ClientError
import pyfolio as pf
import scipy.stats as stats
from numba import jit
from string import Template

isSandbox = False
if isSandbox:
    token = 'Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    #token ='pk_2e94555e43da4135a6032433c6b18fa5'
    token = 'pk_79d147436c1349f3abbec37591323e52'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe = 'max'

/usr/local/lib/python3.7/site-packages/tzlocal/unix.py:177: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn("Can not find any timezone configuration, defaulting to UTC.")
/usr/local/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
def get_ETF_constituents(symbol,etf_holdings_dates_url,etf_holdings_url,schema):
    if isinstance(symbol, str):
        pass
    else:
        symbol=symbol.values[0][0]
         
    cols= [i.split(" ")[0] for i in schema.split(",")]
    vals= ['' if i.split(" ")[1] == 'string' else 0  for i in schema.split(",") ]
    empty_pd= pd.DataFrame([vals],columns=cols)
    findataprep_dates_url = etf_holdings_dates_url.substitute(symbol=symbol)
    empty_pd['etf_symbol']=f"{symbol} : {findataprep_dates_url} : {etf_holdings_url}"
    try:
        dates = requests.get(findataprep_dates_url)
    except  Exception as l_exc:
        ret_pd=empty_pd
        ret_pd['name']=l_exc 
        ret_pd['title']='exception' 
        return(ret_pd)
    all_date_pd = []
    for one_d in dates.json():  
        try:
            findataprep_url = etf_holdings_url.substitute(symbol=symbol,date=one_d.get('date'))
            one_date_json = requests.get(findataprep_url).json()
            one_date_pd=pd.DataFrame(one_date_json)
            one_date_pd['etf_symbol']=symbol
        except:
            one_date_pd=empty_pd
            one_date_pd['etf_symbol']=symbol    
        all_date_pd=all_date_pd+[one_date_pd] 
    if len(all_date_pd)>0:
        ret_pd = pd.concat(all_date_pd)
    else:
        ret_pd=empty_pd
    return(ret_pd)

In [3]:
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','close','open','symbol']
        self.secrets = {
            'IEX':{},
            'alpaca':{ 'token':'alpaca_marketdata_key',
                      'secret':'alpaca_marketdata_secret'}
                       }
        self.data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
        self.parquet_file_out =f"{self.data_folder}/etf_market_data.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            #self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5' pk_79d147436c1349f3abbec37591323e52
            self.iex_token ='pk_79d147436c1349f3abbec37591323e52' 
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        
        self.findataprep_config={}
        self.findataprep_config['etf_holdings_dir']='b753de15443e37f6598e7702811dfad1'
        self.findataprep_config['apikey']='b753de15443e37f6598e7702811dfad1'
        self.findataprep_config['etf_holdings_dates_url']=Template(
            f"https://financialmodelingprep.com/api/v4/etf-holdings/portfolio-date?symbol=$symbol&apikey={self.findataprep_config['apikey']}")
        self.findataprep_config['etf_holdings_url']=Template(
            f"https://financialmodelingprep.com/api/v4/etf-holdings?symbol=$symbol&date=$date&apikey={self.findataprep_config['apikey']}")
        self.findataprep_config['etf_holdings_columns'] = ['cik', 'acceptanceTime', 'date', 'symbol', 'name', 'lei', 'title',
                                                           'cusip', 'isin', 'balance', 'units', 'cur_cd', 'valUsd', 'pctVal',
                                                           'payoffProfile', 'assetCat', 'issuerCat', 'invCountry',
                                                           'isRestrictedSec', 'fairValLevel', 'isCashCollateral',
                                                           'isNonCashCollateral', 'isLoanByFund', 'etf_symbol']
        self.findataprep_config['etf_holdings_column_types']= ['string','string','string',
                                                                'string','string','string','string','string','string',
                                                                'int','string','string','int','float',
                                                                'string','string','string','string','string',
                                                                'string','string','string','string','string',]
        self.findataprep_config['etf_holdings_udf_schema']=",".join([f"{i[0]} {i[1]}" for i in zip( self.findataprep_config['etf_holdings_columns'], self.findataprep_config['etf_holdings_column_types'])])
        self.findataprep_config['etf_holdings_output_file']="s3://fsidatalake/ETF_constituents/etf_holdings.parquet"
        
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema="date timestamp, close double, open double, symbol string")\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
            
    def get_secret(self,secret_name,region_name):
        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

        try:
            get_secret_value_response = client.get_secret_value(
                SecretId=secret_name
            )
        except ClientError as e:
            # For a list of exceptions thrown, see
            # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
            raise e

        # Decrypts secret using the associated KMS key.
        secret = json.loads(get_secret_value_response['SecretString'])[secret_name]
        return(secret)
        

mdg=MarketDataGateway()

In [4]:
etf_holdings_dates_url,etf_holdings_url,schema=mdg.findataprep_config['etf_holdings_dates_url'],mdg.findataprep_config['etf_holdings_url'],mdg.findataprep_config['etf_holdings_udf_schema']
x='SPY'

In [5]:
spy_const= get_ETF_constituents(x,etf_holdings_dates_url,etf_holdings_url,schema)

In [15]:
spy_const.columns

Index(['cik', 'acceptanceTime', 'date', 'symbol', 'name', 'lei', 'title',
       'cusip', 'isin', 'balance', 'units', 'cur_cd', 'valUsd', 'pctVal',
       'payoffProfile', 'assetCat', 'issuerCat', 'invCountry',
       'isRestrictedSec', 'fairValLevel', 'isCashCollateral',
       'isNonCashCollateral', 'isLoanByFund', 'etf_symbol'],
      dtype='object')

In [16]:
spy_const.query('(symbol == "CSGP") or (symbol == "INVH") or (symbol == "PVH") or (symbol == "PENN")')[['date','symbol','acceptanceTime']].drop_duplicates().sort_values(['symbol','date']).style#.query("symbol=='TRGP'")

,date,symbol,acceptanceTime
16,2019-09-30,,2019-11-18 15:09:59
8,2019-12-31,,2020-02-28 14:55:31
2,2020-03-31,,2020-05-29 17:35:52
10,2020-06-30,,2020-08-27 16:57:11
8,2020-09-30,,2020-11-18 20:32:42
17,2020-12-31,,2021-02-26 16:35:15
16,2021-03-31,,2021-05-28 15:06:35
3,2021-06-30,,2021-08-27 14:17:19
20,2021-09-30,,2021-11-24 18:35:01
18,2021-12-31,,2022-02-25 17:20:56
